In [173]:
import requests
import json
import os
import urllib
CONFIG_PATTERN = 'http://api.themoviedb.org/3/configuration?api_key={key}'
IMG_PATTERN = 'http://api.themoviedb.org/3/movie/{imdbid}/images?api_key={key}'
DETAILS_PATTERN = 'https://api.themoviedb.org/3/movie/{imdbid}?api_key={key}'
DISCOVER_PATTERN = 'https://api.themoviedb.org/3/discover/movie?api_key={key}&sort_by=vote_count.desc'

KEY = '<your-Api>'
list_id = '10'
att= '&sort_by=popularity.desc&include_adult=false&include_video=false&page=1'



In [174]:
def _get_json(url):
    r = requests.get(url)
    return r.json()

In [175]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

# Most Voted Movies Details

In [176]:
imdbid = _get_json(DISCOVER_PATTERN.format(key=KEY))['results'][0]['id']

In [177]:
def most_voted_movie_details():
    """ return all details homepage, genders etc.. of the most voted movie 
        Args:
            imdbid (str): IMDB id of the movie
        Returns:
            Details of the movie
    """
    homepage = _get_json(DETAILS_PATTERN.format(key=KEY,imdbid=imdbid))['homepage']
    title = _get_json(DETAILS_PATTERN.format(key=KEY,imdbid=imdbid))['original_title']
    overview = _get_json(DETAILS_PATTERN.format(key=KEY,imdbid=imdbid))['overview']
    releaseDate = _get_json(DETAILS_PATTERN.format(key=KEY,imdbid=imdbid))['release_date']
    revenue = _get_json(DETAILS_PATTERN.format(key=KEY,imdbid=imdbid))['revenue']
    vote_count = _get_json(DETAILS_PATTERN.format(key=KEY,imdbid=imdbid))['vote_count']
    runtime = _get_json(DETAILS_PATTERN.format(key=KEY,imdbid=imdbid))['runtime']
    
    movie_details = "Title : {0} \n\nOverview: \n---------\n {1} \n\nHomepage: \n---------\n {2} \n\nRelease Date: \n---------\n {3}".format(title, overview, homepage, releaseDate)
    
    print(movie_details) 
    

In [178]:
most_voted_movie_details()

Title : Inception 

Overview: 
---------
 Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person's idea into a target's subconscious. 

Homepage: 
---------
 http://inceptionmovie.warnerbros.com/ 

Release Date: 
---------
 2010-07-15


# get IMDb id for movie title

# Download Posters

In [195]:
def get_poster_urls(imdbid):
    """ return image urls of posters for IMDB id
        returns all poster images from 'themoviedb.org'. Uses the
        maximum available size. 
        Args:
            imdbid (str): IMDB id of the movie
        Returns:
            list: list of urls to the images
    """
    config = _get_json(CONFIG_PATTERN.format(key=KEY))
    base_url = config['images']['base_url']
    sizes = config['images']['poster_sizes']

    """
        'sizes' should be sorted in ascending order, so
            max_size = sizes[-1]
        should get the largest size as well.        
    """
    def size_str_to_int(x):
        return float("inf") if x == 'original' else int(x[1:])
    max_size = max(sizes, key=size_str_to_int)

    posters = _get_json(IMG_PATTERN.format(key=KEY,imdbid=imdbid))['posters']
    poster_urls = []
    for poster in posters:
        rel_path = poster['file_path']
        url = "{0}{1}{2}".format(base_url, max_size, rel_path)
        poster_urls.append(url) 

    return poster_urls

In [196]:
get_poster_urls(imdbid)

['http://image.tmdb.org/t/p/original/tAXARVreJnWfoANIHASmgYk4SB0.jpg',
 'http://image.tmdb.org/t/p/original/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg',
 'http://image.tmdb.org/t/p/original/oFsUrLa9u4jLEshKDgWXADPy4kG.jpg',
 'http://image.tmdb.org/t/p/original/ag5mtLNWp1zDqNxZF6Lnk87JknW.jpg',
 'http://image.tmdb.org/t/p/original/wlrjaMWMP8gpELu2oFNW0VJHEy1.jpg',
 'http://image.tmdb.org/t/p/original/9Bb4zpZK6N9I0rcaMNvTIotWCX0.jpg',
 'http://image.tmdb.org/t/p/original/n9dwu1p5G4qJ4DI5eHJMUbAdOfA.jpg',
 'http://image.tmdb.org/t/p/original/e8MyVAjISnQ8TCcOPQ0Pk2PTe46.jpg',
 'http://image.tmdb.org/t/p/original/vZ9B2dp2MQrFCcIOwsioYHQUa1a.jpg',
 'http://image.tmdb.org/t/p/original/kTzw1Mt0kf15SAOZzU3j1bPhv2n.jpg',
 'http://image.tmdb.org/t/p/original/lbFaJLrR0DnvVGbDswv865gSMeT.jpg',
 'http://image.tmdb.org/t/p/original/r84x4x93LbZ2gozISTBYVeq0gLZ.jpg',
 'http://image.tmdb.org/t/p/original/yTtFJnaYQuFPetgwsSD2Ig3SjsO.jpg',
 'http://image.tmdb.org/t/p/original/lxbTzHi4qChQSUJLHYXc5ply4ca.jpg',
 'http

In [197]:
def _download_images(urls, path='.\poster'):
    """download all images in list 'urls' to 'path' """

    for nr, url in enumerate(urls):
        r = requests.get(url)
        filetype = r.headers['content-type'].split('/')[-1]
        filename = 'poster_{0}.{1}'.format(nr+1,filetype)
        filepath = os.path.join(path, filename)
        with open(filepath,'wb') as w:
            w.write(r.content)

In [198]:
def tmdb_posters(imdbid, count=None, outpath='.\poster'):    
    urls = get_poster_urls(imdbid)
    if count is not None:
        urls = urls[:count]
    _download_images(urls, outpath)


In [199]:
tmdb_posters(imdbid)

KeyboardInterrupt: 